<a href="https://colab.research.google.com/github/kovzanok/dls-final-task/blob/main/2_ArcFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Зависимости и загрузка данных

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/file.zip -d /content/

In [ ]:
from torch import nn
from torch.nn import functional as F
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from PIL import Image
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torchvision.models as models
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import torchvision.transforms.functional as TF
import cv2
from sklearn.preprocessing import LabelEncoder


import random
import os
import math

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

#Датасет

Загружаем датасет со связью имени изображения и id человека

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/identity_CelebA.txt', delim_whitespace=True, header=None,index_col=0)

In [ ]:
df.index.name = 'image_name'
df.rename(columns={1:'id'},inplace=True)

In [ ]:
df.head()

In [ ]:
def get_filenames_os(folder_path):
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    return files

Получаем имя файлов из датасета, прошедшего выравнивание и кроп моделью из прошлого задания. Если бы использовался датасет CelebA полностью эта операция не нужна была бы, но нам нужны id только тех людей, фото которых мы обработали в прошлом задании.

In [ ]:
filenames = get_filenames_os('/content/content/stage_3_dataset')

In [ ]:
filtered_df = df.loc[filenames]


Кодируем id людей, т.к. они не упорядочены

In [ ]:
le = LabelEncoder()
filtered_df['encoded_id'] = le.fit_transform(filtered_df['id'])

Кастомный датасет, возвращающий изображение и закодированный id

In [ ]:
class CelebADataset(Dataset):
    def __init__(self, img_dir, img_names, image_df, transform=None):
        super().__init__()
        self.img_dir = img_dir
        self.transform = transform
        self.image_names = img_names
        self.image_df = image_df

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        id = self.image_df.loc[img_name]['encoded_id']

        return image, int(id)


В финальной версии применен ряд аугментаций, повысивших итоговый accuracy. При первом обучении из аугментаций был только `Resize` и `Normalize`

In [ ]:
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomAffine(degrees=5, translate=(0.02, 0.02)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3),
])


dataset = CelebADataset(
    img_dir='/content/content/stage_3_dataset',
    transform=transform,
    image_df=filtered_df,
    img_names = filtered_df.index.to_list()
)

In [ ]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=128)

#Модель

Предобученная модель resnet18 с замененым последним слоем, количество нейронов на конце равно количеству классов(около 10к для нашего датасета)

In [17]:
num_classes = int(filtered_df.nunique().values[0])

In [18]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 186MB/s]


Загружаем сохраненную ранее лучшую версию модели

In [19]:
model.load_state_dict(torch.load('/content/drive/MyDrive/best_recognition_model.pth'))

<All keys matched successfully>

В начале обучения разморожен был только `layer4` и `fc`. Далее после достижения "потолка" accuracy было принято решение разморозить `layer3` для его файн-тюнинга

In [20]:
for name, param in model.named_parameters():
    if 'layer3' in name or 'layer4' in name or 'fc' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False


In [21]:
model = model.to(device)

#Обучение

##Функции обучения

In [ ]:
def train_epoch(model, classifier, criterion, optimizer, dataloader):
    model.train()
    if classifier is not None:
        classifier.train()

    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(dataloader, leave=False):
        images, labels = images.to(device), labels.to(device)

        features = model(images)  # эмбеддинги

        if classifier is not None:
            logits = classifier(features, labels)
        else:
            logits = features  # если model уже даёт logits

        loss = criterion(logits, labels)

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(logits, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = running_loss / len(dataloader)
    accuracy = correct / total
    return avg_loss, accuracy


In [ ]:
@torch.no_grad()
def val_epoch(model, classifier, criterion, dataloader):
    model.eval()
    if classifier is not None:
        classifier.eval()

    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(dataloader, leave=False):
        images, labels = images.to(device), labels.to(device)

        features = model(images)

        if classifier is not None:
            logits = classifier(features, labels)
        else:
            logits = features

        loss = criterion(logits, labels)
        running_loss += loss.item()

        _, preds = torch.max(logits, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = running_loss / len(dataloader)
    accuracy = correct / total
    return avg_loss, accuracy


In [ ]:
def train(model, criterion, optimizer, train_loader, val_loader, num_epochs=100, scheduler=None, classifier=None):
    history = {'train':[],'test':[],'accuracy':[]}
    best_val_loss = float('inf')
    best_accuracy = 0

    for epoch in tqdm(range(num_epochs)):
        train_loss, train_acc = train_epoch(model, classifier, criterion, optimizer, train_loader)
        val_loss, val_acc = val_epoch(model, classifier, criterion, val_loader)

        history['train'].append(train_loss)
        history['test'].append(val_loss)
        history['accuracy'].append(val_acc)

        print(f"Epoch {epoch + 1} — Train Loss: {train_loss:.4f} — Val Loss: {val_loss:.4f} - Accuracy on val: {val_acc}")

        if scheduler:
          scheduler.step(val_acc)

        if val_acc > best_accuracy:
            best_accuracy = val_acc
            torch.save({
                'model': model.state_dict(),
                'classifier': classifier.state_dict() if classifier else None
            }, "best_model.pth")
            print("✅ Saved best model")


    return history

##Cross Enthropy Loss

Тут уже ChatGPT подсказал, что `label_smoothing` аргумент для CE loss'а может быть полезен для борьбы с переобучением и делает границы классификации более "мягкими"

In [ ]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.05)
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

Классический цикл обучения с итерацией по эпохам и датасету. Сохранение лучшей модели происходит после каждой эпохи, если полученная accuracy выше предыдущего сохраненного значения

In [ ]:
history = train(model, criterion, optimizer, train_loader, val_loader, scheduler=scheduler)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/950 [00:00<?, ?it/s]

  0%|          | 0/238 [00:00<?, ?it/s]

Epoch 1 — Train Loss: 1.4193 — Val Loss: 2.7241 - Accuracy on val: 0.6310957551826258
✅ Model saved (best so far)


  0%|          | 0/950 [00:00<?, ?it/s]

...

  0%|          | 0/950 [00:00<?, ?it/s]

  0%|          | 0/238 [00:00<?, ?it/s]

Epoch 11 — Train Loss: 1.1026 — Val Loss: 2.5973 - Accuracy on val: 0.6573872984534387


  0%|          | 0/950 [00:00<?, ?it/s]

KeyboardInterrupt: 

Обучение на CE loss было реализовано поэтапно. Исходные данные: модель resnet18, предобученная на ImageNet, с размороженным layer4. Для обучения было взято 25% от датасета CelebA, оптимизатор Adam, без scheduler. По итогам обучения в течение +-30 эпох было получено accuracy около 0.11. Модель была сохранена для будущего дообучения. Далее будут приведены шаги по улучшению процесса обучения и их результаты(все шаги были пройдены на модели, которая сохранялась на шаге и загружалась в следующий):
1. Увеличил размер датасета до 75% от исходного и добавил аугментаций. После +-30 эпох accuracy достигло 0.33
2. Разморозил помимо layer4 еще и layer3, уменьшил learning_rate до `1e-4`, добавил scheduler ReduceLROnPlateau, но его настроил на `mode='min'` и передавал в метод `scheduler.step(val_loss)`. После 10 эпох accuracy 0.66
3. Scheduler изменил на `mode='max'` и `scheduler.step(accuracy)`. В итоге удалось добить accuracy до 0.84

##Arc Face Loss

In [ ]:
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, s=30.0, m=0.5):
        super().__init__()
        self.s = s
        self.m = m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # L2 normalize input and weights
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.clamp(cosine**2, 0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m  # cos(θ + m)

        # one-hot encode labels
        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, label.view(-1, 1), 1.0)

        # apply arc margin only to target class
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)

        # scale logits
        output *= self.s
        return output


In [ ]:
backbone = models.resnet18(pretrained=True)
backbone.fc = nn.Identity()
model = nn.Sequential(backbone, nn.Flatten()).to(device)

arc_margin = ArcMarginProduct(in_features=512, out_features=num_classes).to(device)


for name, param in model.named_parameters():
    if 'layer4' in name or 'fc' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([
    {'params': model.parameters()},
    {'params': arc_margin.parameters()}
], lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

In [40]:
history = train(model, criterion, optimizer, train_loader, val_loader, scheduler=scheduler, classifier=arc_margin)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/950 [00:00<?, ?it/s]

  0%|          | 0/238 [00:00<?, ?it/s]

Epoch 1 — Train Loss: 98.7772 — Val Loss: 92.2252 - Accuracy on val: 0.0


  0%|          | 0/950 [00:00<?, ?it/s]

...

  0%|          | 0/950 [00:00<?, ?it/s]

  0%|          | 0/238 [00:00<?, ?it/s]

Epoch 15 — Train Loss: 89.6055 — Val Loss: 89.5910 - Accuracy on val: 0.0


  0%|          | 0/950 [00:00<?, ?it/s]

KeyboardInterrupt: 